In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
import torch
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as ConstantKernel
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale

# Load data

In [2]:
# load data
data = pd.read_excel('C:/Users/CHENM0D/Desktop/Genetic Algorithm/Ensemble learning/Original data/training dataset.xlsx')
temperature_index = ['T']
y = data[temperature_index]
print(y.describe())
# print(y.mean(axis=0))

fthick = ['fthick']
thick = data[fthick]
print(thick.mean(axis=0))

scaler = StandardScaler()
scaler.fit(y)
print(scaler.mean_)
print(scaler.var_)
T_norm = scaler.transform(y)


# Compare with jenny's data
data2 = pd.read_excel('C:/Users/CHENM0D/Desktop/Genetic Algorithm/Python program-Jenny/data_scaled.xlsx')
y_index = ['T']
T_norm2 = data2[y_index]

out = T_norm-T_norm2
out.sum(axis=0)

# outcome shows exactly the same

                T
count  692.000000
mean   246.504335
std    100.398936
min     25.000000
25%    163.750000
50%    296.000000
75%    298.000000
max    600.000000
fthick    85.762139
dtype: float64
[246.50433526]
[10065.38003901]


T   -7.993606e-15
dtype: float64

# Rank data based on T

In [3]:
new_y = y.rename(columns={'T':'TO'})
data3 = pd.concat([data2, new_y], join = 'outer', axis = 1)
data3

,T,fmass,fEb,sEb,smass,fthick,fmelt,fdensity,sdensity,fAC1x,...,fENc,fENa,sheatcap,sunit,sR1,sR2,sENc,sENa,itr,TO
0,-1.460277,1.140231,0.073797,0.737867,-0.569384,0.395161,0.254254,0.88601,0.033333,0.666667,...,2.54,2.54,0.275127,2.003,1,1,1.90,1.90,26.315789,100
1,-0.961904,1.140231,0.073797,0.737867,-0.569384,0.395161,0.254254,0.88601,0.033333,0.666667,...,2.54,2.54,0.443583,2.003,1,1,1.90,1.90,24.390244,150
2,-0.463531,1.140231,0.073797,0.737867,-0.569384,0.395161,0.254254,0.88601,0.033333,0.666667,...,2.54,2.54,0.570319,2.003,1,1,1.90,1.90,25.641026,200
3,0.034843,1.140231,0.073797,0.737867,-0.569384,0.395161,0.254254,0.88601,0.033333,0.666667,...,2.54,2.54,0.656553,2.003,1,1,1.90,1.90,22.727273,250
4,0.493347,1.140231,0.073797,0.737867,-0.569384,0.395161,0.254254,0.88601,0.033333,0.666667,...,2.54,2.54,0.710557,2.003,1,1,1.90,1.90,21.276596,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,-0.662880,-0.478273,0.211788,-1.897606,1.276872,0.495968,0.260995,0.34715,0.458974,0.666667,...,1.90,1.90,0.410000,4.247,2,3,1.61,3.44,7.142857,180
688,-0.862229,-0.478273,0.211788,-1.897606,1.276872,0.495968,0.260995,0.34715,0.458974,0.666667,...,1.90,1.90,0.320000,4.247,2,3,1.61,3.44,10.000000,160
689,-1.260928,-0.478273,0.211788,-1.897606,1.276872,0.495968,0.260995,0.34715,0.458974,0.666667,...,1.90,1.90,0.210000,4.247,2,3,1.61,3.44,11.111111,120
690,-1.510115,-0.478273,0.211788,-1.897606,1.276872,0.495968,0.260995,0.34715,0.458974,0.666667,...,1.90,1.90,0.110000,4.247,2,3,1.61,3.44,14.285714,95


In [4]:
data3.sort_values('TO')

,T,fmass,fEb,sEb,smass,fthick,fmelt,fdensity,sdensity,fAC1x,...,fENc,fENa,sheatcap,sunit,sR1,sR2,sENc,sENa,itr,TO
306,-2.207838,1.140231,0.073797,-1.897606,1.276872,0.445565,0.254254,0.886010,0.458974,0.666667,...,2.54,2.54,0.001000,4.247,2,3,1.61,3.44,83.333333,25
460,-2.058326,-0.921900,0.250598,0.737867,-0.569384,0.495968,0.124880,0.025907,0.033333,0.833333,...,1.61,1.61,0.051178,2.003,1,1,1.90,1.90,29.411765,40
313,-1.958651,1.140231,0.073797,0.472989,-0.971276,0.445565,0.254254,0.886010,0.335897,0.666667,...,2.54,2.54,0.002111,0.568,1,1,2.55,2.55,100.000000,50
350,-1.958651,-0.668485,-0.357427,-1.897606,1.276872,0.445565,0.452006,0.119171,0.458974,0.083333,...,1.54,1.54,0.015000,4.247,2,3,1.61,3.44,25.000000,50
461,-1.958651,-0.921900,0.250598,0.737867,-0.569384,0.495968,0.124880,0.025907,0.033333,0.833333,...,1.61,1.61,0.088055,2.003,1,1,1.90,1.90,15.873016,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,2.526710,-0.921900,0.250598,0.737867,-0.569384,0.294355,0.124880,0.025907,0.033333,0.833333,...,1.61,1.61,0.817617,2.003,1,1,1.90,1.90,8.620690,500
451,2.576548,1.117304,-0.771402,0.065198,-0.246970,0.243952,0.481541,1.000000,0.282051,0.583333,...,2.28,2.28,0.995467,2.085,1,1,1.61,3.04,5.882353,505
562,3.025084,-0.921900,0.250598,-0.039674,-0.269214,0.279234,0.124880,0.025907,0.256410,0.833333,...,1.61,1.61,1.004644,2.059,1,1,1.90,2.55,5.988024,550
568,3.025084,-0.921900,0.250598,0.737867,-0.569384,0.294355,0.124880,0.025907,0.033333,0.833333,...,1.61,1.61,0.829125,2.003,1,1,1.90,1.90,9.803922,550


# Construct material systems

In [5]:
fheatcap = data['fheatcap']
sheatcap = data['sheatcap']
fheatcap.describe()

count    692.000000
mean       0.435874
std        0.317988
min        0.000000
25%        0.124317
50%        0.361659
75%        0.801415
max        0.899901
Name: fheatcap, dtype: float64

In [6]:
sheatcap.describe()

count    692.000000
mean       0.530202
std        0.260197
min        0.001000
25%        0.319751
50%        0.594501
75%        0.728511
max        1.050000
Name: sheatcap, dtype: float64

In [8]:
#load descriptor data
des_data = pd.read_excel('C:/Users/CHENM0D/Desktop/Genetic Algorithm/Ensemble learning/Original data/descriptor dataset.xlsx')

In [9]:
des_data

,id-M,Material,Formula,Specific Heat capacity (J/gK),Melting point (K),Density (g/cm3),Volume per formula unit (10-29 m3/f.u.),R1,R2,Mass (u),AC1x,AC1y,AC2x,AC2y,ENc,ENa,IPc,IPa,Eb(eV/f.u.)
0,1,Silicon,Si,0.710000,1687.0,2.330,2.003000,1,1,28.0855,14.00,3,14,3,1.90,1.90,8.151,8.151,-4.622464
1,2,Germanium,Ge,0.310000,1211.0,5.340,2.256000,1,1,72.6400,14.00,4,14,4,2.01,2.01,7.899,7.899,-3.855508
2,3,Glass,a-SiO2,0.750000,1873.0,2.200,4.533634,1,2,60.0843,14.00,3,16,2,1.90,3.44,8.151,13.618,-19.647327
3,4,Gold,Au,0.130000,1337.0,19.300,1.696000,1,1,196.9665,11.00,6,11,6,2.54,2.54,9.225,9.225,-3.814051
4,5,Aluminium,Al,0.900000,934.0,2.700,1.650250,1,1,26.9815,13.00,3,13,3,1.61,1.61,5.986,5.986,-3.389116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,334,Titanium Nitride,TiN,0.601020,3608.0,5.232,1.910000,1,1,61.8737,4.00,4,15,2,1.54,3.04,6.820,14.534,-13.577141
294,335,Uranium Mononitride,UN,0.180600,3078.0,14.320,2.921750,1,1,252.0356,3.04,7,15,2,1.38,3.04,6.050,14.534,-13.469911
295,336,Vanadium Nitride,VN,0.587160,2392.0,6.170,1.748250,1,1,64.9482,5.00,4,15,2,1.63,3.04,6.740,14.534,-12.547169
296,337,Zirconium Nitride,ZrN,0.385686,3093.0,7.200,2.390000,1,1,105.2307,4.00,5,15,2,1.33,3.04,6.840,14.534,-14.902317


In [10]:
des_data.describe()

,id-M,Specific Heat capacity (J/gK),Melting point (K),Density (g/cm3),Volume per formula unit (10-29 m3/f.u.),R1,R2,Mass (u),AC1x,AC1y,AC2x,AC2y,ENc,ENa,IPc,IPa,Eb(eV/f.u.)
count,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000
mean,167.063758,0.516601,1704.932047,6.157876,5.303125,1.395973,2.110738,187.000916,7.186309,4.614094,14.711409,2.778523,1.647685,2.810805,7.155550,11.722893,-16.892766
std,97.483795,0.352920,901.160012,3.511985,3.621906,0.890041,1.767025,147.093974,4.495264,1.220781,3.274831,1.109164,0.418794,0.693186,1.351615,2.869849,12.827486
min,1.000000,0.049560,146.000000,0.780000,0.563750,1.000000,1.000000,7.948900,1.000000,2.000000,1.000000,1.000000,0.790000,1.310000,3.894000,5.786000,-106.236010
25%,82.250000,0.292761,976.000000,3.590000,2.739250,1.000000,1.000000,91.644600,3.085000,4.000000,14.000000,2.000000,1.337500,2.115000,6.085250,9.009000,-21.443666
50%,171.000000,0.450000,1646.000000,5.435000,4.376250,1.000000,2.000000,152.718700,6.000000,4.000000,16.000000,2.000000,1.630000,2.660000,7.099000,11.814000,-13.517015
75%,248.750000,0.661815,2297.750000,7.517500,6.790188,1.000000,3.000000,236.983675,11.000000,6.000000,16.000000,3.000000,1.910000,3.440000,7.870000,13.618000,-7.922568
max,338.000000,3.508260,4510.000000,22.550000,26.280000,7.000000,13.000000,1022.595000,18.000000,7.000000,17.000000,6.000000,2.600000,3.980000,12.130000,17.422000,-1.502819


In [11]:
a = des_data[:2]
a = a.rename(columns=lambda s: 'f'+s)
a.reset_index(drop=True, inplace=True)
print(a.shape)
a

(2, 19)


,fid-M,fMaterial,fFormula,fSpecific Heat capacity (J/gK),fMelting point (K),fDensity (g/cm3),fVolume per formula unit (10-29 m3/f.u.),fR1,fR2,fMass (u),fAC1x,fAC1y,fAC2x,fAC2y,fENc,fENa,fIPc,fIPa,fEb(eV/f.u.)
0,1,Silicon,Si,0.71,1687.0,2.33,2.003,1,1,28.0855,14.0,3,14,3,1.90,1.90,8.151,8.151,-4.622464
1,2,Germanium,Ge,0.31,1211.0,5.34,2.256,1,1,72.6400,14.0,4,14,4,2.01,2.01,7.899,7.899,-3.855508


In [ ]:
all_entries = []
for i in range(298):
    for j in range(298):
        a = des_data[i:i+1]
        a = a.rename(columns=lambda s: 'f'+s)
        a.reset_index(drop=True, inplace=True)
        if i!=j:
            b = des_data[j:j+1]
            b = b.rename(columns=lambda s: 's'+s)
            b.reset_index(drop=True, inplace=True)
            c = pd.concat([a,b], axis=1)
            all_entries.append(c)
        else:
            continue
print(len(all_entries))

In [ ]:
ms = all_entries[0]
for i in range(1,len(all_entries)):
    ms = pd.concat([ms, all_entries[i]], axis=0)
ms
    
    

In [ ]:
ms.to_excel("output.xlsx")

In [ ]:
print('finished')

In [ ]:
test_data = pd.read_excel('C:/Users/Mingguang Chen/Desktop/Genetic Algorithm/Ensemble learning/Original data/test data - Copy.xlsx')